In [1]:
from shared import *
from utils.wearable_fit import get_all_gotchi_wearable_fit_counts_by_slot_rarity_df, get_changes_to_rarity_slot_fit_count_stddev_df, get_changes_to_set_wearable_fit_count_stddev_df
from utils.wearable_sets import filter_wearable_sets_by_rarity_range, get_wearable_sets_df

wearable_types_df = get_wearable_types_df()
wearable_sets_filtered_df = filter_wearable_sets_by_rarity_range(get_wearable_sets_df(wearable_types_df), wearable_types_df)

In [2]:
gotchi_wearable_fit_counts_slot_rarity_df = get_all_gotchi_wearable_fit_counts_by_slot_rarity_df(wearable_types_df)

In [3]:
changes_to_rarity_slot_fit_count_stddev_df = get_changes_to_rarity_slot_fit_count_stddev_df(wearable_types_df)         
changes_to_rarity_slot_fit_count_stddev_df.sort_values(by=['rarity_sort', 'slot', 'trait_modifier_count', 'change_in_rarity_slot_fit_count_stddev'], ascending=True, inplace=True)

# Top Trait Modifier Choices for New Schematics

Use this table to identify suitable trait modifier combinations for newly released wearable schematics given a particular rarity and slot.

This table shows the top choices (up to 5) for trait modifier combinations by slot and rarity using the trait selection methodology described in the [Wearable Trait Selection: A Quantitative Approach](https://docs.google.com/document/d/1D9e2FEh5NnYXrPEK28UF7GWpsI1ybOsCY94x1kGCpJs/edit?usp=drive_link) litepaper. The respective delta standard deviations are shown for each combination (lower numbers are better). Any choices that would negatively impact meta balance (delta standard deviation > 0) have been omitted.



In [4]:
NUM_CHOICES_TO_SHOW = 5

def get_top_n_rows(df, group_col, value_col, n, ascending=True):
    top_n_rows = df.groupby(group_col, as_index=False).apply(lambda x: x.sort_values(value_col, ascending=ascending).head(n))
    top_n_rows.reset_index(drop=True, inplace=True)
    return top_n_rows

new_trait_top_choices_df = get_top_n_rows(changes_to_rarity_slot_fit_count_stddev_df[changes_to_rarity_slot_fit_count_stddev_df['change_in_rarity_slot_fit_count_stddev'] < 0], ['rarity', 'slot', 'rarity_sort'], 'change_in_rarity_slot_fit_count_stddev', NUM_CHOICES_TO_SHOW)
new_trait_top_choices_df['change_in_rarity_slot_fit_count_stddev'] = new_trait_top_choices_df['change_in_rarity_slot_fit_count_stddev'].apply(lambda x: round(x, 2))
new_trait_top_choices_df = new_trait_top_choices_df.groupby(['rarity', 'slot', 'rarity_sort']).agg({'trait_modifiers_to_add': list, 'change_in_rarity_slot_fit_count_stddev': list}).reset_index()
order = new_trait_top_choices_df.sort_values(by=['rarity_sort', 'slot'], ascending=True, inplace=True)
show_itable(new_trait_top_choices_df[['rarity', 'slot', 'trait_modifiers_to_add', 'change_in_rarity_slot_fit_count_stddev']].reset_index(drop=True), order=[])

# Top Trait Combinations for New Sets

Use this table to determine which trait combinations fit best together in a new 3-piece set, for set balance purposes. Calculation is done by counting the number of unique wearables each gotchi could equip in a perfectly fitting set (all traits match for set bonuses and wearables). Lesser used sets (for example, sets with a high rarity spread- combining godlike with common wearables, etc) are excluded from the calculation to better reflect the usefulness of each gotchi's current sets.

In [5]:
changes_to_set_wearable_fit_count_stddev_df = get_changes_to_set_wearable_fit_count_stddev_df(wearable_types_df, wearable_sets_filtered_df, num_pieces_in_set=3)
show_itable_long(changes_to_set_wearable_fit_count_stddev_df, order=[[2, 'asc']], column_formats={'change_in_fit_count_stddev': '{:.2f}'})